# Derivatives-Oriented Self-Supervision

The following notebook will accompany my research.  
In this research, I'd like to introduce a novel framework to recognize ODE from data.  
In further steps, I'll compare my method with the state-of-the-art deep learning model.

# 0. Imports 

In [1]:
import numpy as np
import scipy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

sns.set(style='darkgrid')
%matplotlib ipympl

np.random.seed(42)

# 1. Data Generation 

In this section I'll generate a data tables for the experiments.  
I'll use the a simple ODE the get data:  
$$
\frac{dy}{dx} = y + x^2
$$
I'll solve this equation using Runge-Kutta 4th order method:  
For $$ 
\frac{dy}{dx} = f(x_i, y_i)  
$$
The solution calculated as follow:
$$
K_1 = hf(x_i, y_i) \\
K_2 = hf(x_i + \frac{h}{2}, y_i + \frac{k_1}{2})  \\  
K_3 = hf(x_i, y_i + \frac{k_2}{2})  \\
K_4 = hf(x_i, y_i + k_3) \\
y_{i+1} = y_i + \frac{k_1}{6} + \frac{k_2}{3} + \frac{k_3}{3} + \frac{k_4}{6} + O(h^5)  \\
$$

In [2]:
def f(x, y):
    return y + x ** 2

def runge_kutta(x0, y0, x, h):
    y = y0
    steps = int((x - x0) / h)
    for i in range(0, steps):
        k1 = h * f(x, y)
        
        k2 = h * f(x + (h / 2), y + k1 / 2)
        
        k3 = h * f(x + (h / 2), y + k2 / 2)
        
        k4 = h * f(x + h, y + k3)
        
        y = y + k1/6 + k2/3 + k3/3 + k4/6
        x0 = x0 + h
    
    return y

In [3]:
x0 = 0
y0 = 3
h = 0.1
x1 = 1

runge_kutta(x0, y0, x1, h)

10.047666545097762

In [4]:
x1 = np.linspace(0, 10, 100)
h1 = 0.1
y1 = [3]


for i in range(len(x1) - 1):
    y_next = runge_kutta(x1[i], y1[i], x1[i+1], h1)
    y1.append(y_next)

Now I'll create another track with different initial values for comparison:

In [5]:
x2 = np.linspace(0, 10, 100)
h2 = 0.1
y2 = [6]

for i in range(len(x2) - 1):
    y_next = runge_kutta(x2[i], y2[i], x2[i+1], h2)
    y2.append(y_next)

In [6]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

subsut_size=20

fig = plt.figure()
ax = fig.add_subplot(1,2,1)
ax.scatter(x1[:subsut_size],y1[:subsut_size])
ax = fig.add_subplot(1,2,2)
ax.scatter(x2[:subsut_size],y2[:subsut_size])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We've got 2 sets of points, each set represents different initial values.  
In order to determine if those sets follow the same D-th order ODE, we want to find linear combination such that:  
$$ \sum_{i=0}^D{\alpha_i y^{(i)}_j} : j \in [0,N] | y_j = y(x_j) $$  
Thus, we can use a regression model (_Linear-Regression_ for linear ODEs, or DNNs elsewhere) to find those $ \alpha $s.  
Then, we can use those weights to feed-forward the other set, and assume to get all the values to be 0 ($\pm \epsilon$).

Now, let's create a derivative function:  
Assume our data created by some: $ y = f(x_1,x_2, ... ,x_n) $, we'd like to calculate the following partial derivatives:  
$$
\frac{\partial z}{\partial x_i} = \lim_{h\to 0}\frac{f(x_1, ..., x_i + h, ..., x_n) - f(x_1, ..., x_i, ..., x_n)}{h}
$$  


In [7]:
def derivative(x, y, i):
    """
    Assume x, y,  sorted by x
    return f'(x_i, y_i)
    """
    return (y[i+1] - y[i]) / (x[i+1] - x[i])

In [8]:
x1 = np.linspace(0, 10, 1000)
h1 = 0.1
y1 = [3]


for i in range(len(x1) - 1):
    y_next = runge_kutta(x1[i], y1[i], x1[i+1], h1)
    y1.append(y_next)

In [9]:
x2 = np.linspace(0, 10, 1000)
h2 = 0.1
y2 = [6]

for i in range(len(x2) - 1):
    y_next = runge_kutta(x2[i], y2[i], x2[i+1], h2)
    y2.append(y_next)

In [10]:
y1_d1 = [derivative(x1, y1, i) for i in range(len(x1) - 1)]
y1_d2 = [derivative(x1, y1_d1, i) for i in range(len(x1) - 2)]
y1_d3 = [derivative(x1, y1_d2, i) for i in range(len(x1) - 3)]

y2_d1 = [derivative(x2, y2, i) for i in range(len(x2) - 1)]
y2_d2 = [derivative(x2, y2_d1, i) for i in range(len(x2) - 2)]
y2_d3 = [derivative(x2, y2_d2, i) for i in range(len(x2) - 3)]

In [11]:
x_train1 = np.vstack([y1[:-3], y1_d1[:-2], y1_d2[:-1], y1_d3]).T
x_train2 = np.vstack([y2[:-3], y2_d1[:-2], y2_d2[:-1], y2_d3]).T

y_train1 = np.zeros(x_train1.shape[0])
y_train2 = np.zeros(x_train2.shape[0])
# y_train1 = np.random.normal(scale=1e-5,size=x_train1.shape[0])

In [12]:
x_train = np.vstack((x_train1, x_train2))
y_train = np.hstack((y_train1, y_train2))

In [13]:
print(x_train.shape)
x_train1.shape, x_train2.shape

(1994, 4)


((997, 4), (997, 4))

While most of the self-supervised learning implementations over sequences problems assuming auto-regressive model:
x_{t+1} =f(x_t,x_{t-1} ,…,x_{t-p} )
to predict following parts of the data (words, pixels, graph edges etc. [5]), we'll use subsets of the data to form a differential equation that fits to the observation's behavior. 

In [14]:
from sklearn.linear_model import LinearRegression

def train_lr(x, y):
    lr = LinearRegression(fit_intercept=True).fit(x, y)
    y_pred = lr.predict(x)
    print(f'Score: {lr.score(x, y)}')
    print(f'coefficients: {lr.coef_}')
    return lr
    
train_lr(x_train, y_train)

Score: 1.0
coefficients: [0. 0. 0. 0.]


LinearRegression()

Now, let's try DNN:

In [15]:
import torch
import torch.nn as nn

class RegressionNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressionNet, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.activation = nn.ReLU()
        self.hidden = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.activation(x)
        x = self.hidden(x)
        return x

In [16]:
input_size = x_train.shape[1]
hidden_size = 20
output_size = 1
model = RegressionNet(input_size, hidden_size, output_size)

In [17]:
def to_torch(x_train, y_train):
    X = torch.from_numpy(x_train.astype(np.float32))
    y = torch.from_numpy(y_train.astype(np.float32))
    y = y.view(y.shape[0], 1)
    return X, y

In [18]:
def train_net(model, x_train, y_train):
    # hyperparameters
    learning_rate = 0.01
    n_iters = 100

    loss = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    X, y = to_torch(x_train, y_train)

    for epoch in range(n_iters):
        # predict = forward pass with our model
        y_predicted = model(X)

        # loss
        l = loss(y_predicted, y)

        # calculate gradients = backward pass
        l.backward()

        # update weights
        optimizer.step()

        # zero the gradients after updating
        optimizer.zero_grad()

        if (epoch+1) % 10 == 0:
    #         w, b = model.parameters()
            print('epoch ', epoch+1,' loss = ', l.item())
train_net(model, x_train, y_train)

epoch  10  loss =  0.0026805633679032326
epoch  20  loss =  0.002501153852790594
epoch  30  loss =  0.0023337078746408224
epoch  40  loss =  0.0021774256601929665
epoch  50  loss =  0.002031564712524414
epoch  60  loss =  0.001895430264994502
epoch  70  loss =  0.0017683719052001834
epoch  80  loss =  0.0016497913748025894
epoch  90  loss =  0.0015391299966722727
epoch  100  loss =  0.0014358540065586567


In [19]:
y_predicted = model(to_torch(x_train,  y_train)[0])
y_predicted.mean()

tensor(-0.0116, grad_fn=<MeanBackward0>)

In [20]:
model.hidden.weight

Parameter containing:
tensor([[ 0.0399,  0.0569, -0.0238,  0.0778, -0.2066,  0.1162, -0.1178, -0.2002,
         -0.2036,  0.0946,  0.1467,  0.0899,  0.1924, -0.0259, -0.1464, -0.1230,
         -0.0489,  0.1050, -0.2097,  0.0609]], requires_grad=True)

Great! now we got some non-trivial solution.  
Let's try predict one set by learning the other:

In [21]:
x3 = np.linspace(0, 10, 1000)
h3 = 0.1
y3 = [6]

for i in range(len(x3) - 1):
    y_next = runge_kutta(x3[i], y3[i], x3[i+1], h3)
    y3.append(y_next)

In [22]:
y3_d1 = [derivative(x3, y3, i) for i in range(len(x3) - 1)]
y3_d2 = [derivative(x3, y3_d1, i) for i in range(len(x3) - 2)]
y3_d3 = [derivative(x3, y3_d2, i) for i in range(len(x3) - 3)]

In [23]:
x_train3 = np.vstack([y3[:-3], y3_d1[:-2], y3_d2[:-1], y3_d3]).T
y_train3 = np.zeros(x_train3.shape[0])

x_train3 = torch.from_numpy(x_train3.astype(np.float32))
y_train3 = torch.from_numpy(y_train3.astype(np.float32))
y_train3 = y_train3.view(y_train3.shape[0], 1)

In [24]:
def print_loss(model, x, y):
    y_predicted = model(x)
    l = nn.MSELoss()(y_predicted, y)
    print(l.item())
print_loss(model, x_train3, y_train3)

0.0005894683999940753


Now, let's repeat the process using exact solutions: $y(x) = ce^x - x^2 -2x - 2$

In [25]:
def y_exact(x, c):
    return c * np.exp(x) - x ** 2 - 2*x - 2

In [26]:
x_exac = [np.sort(np.random.random(1000) * 10) for i in range(3)]
c = [1, 3, 6]
y_exac = [[y_exact(ci, x_i) for x_i in x_exac_i] for ci, x_exac_i in zip(c, x_exac) ]

In [40]:
def gather_derivatives(x, y):
    d1 = [derivative(x, y, i) for i in range(len(x) - 1)]
    d2 = [derivative(x, d1, i) for i in range(len(x) - 2)]
    d3 = [derivative(x, d2, i) for i in range(len(x) - 3)]
    x_ret = np.vstack([y[:-3], d1[:-2], d2[:-1], d3]).T
    return x_ret

In [42]:
x_exac_train1, x_exac_train2, x_exac_train3 = \
    [gather_derivatives(x, y) for x, y in zip(x_exac, y_exac)]

y_exac_train1 = np.zeros(x_train1.shape[0])
y_exac_train2 = np.zeros(x_train2.shape[0])
y_exac_train3 = np.zeros(x_train3.shape[0])
# y_train1 = np.random.normal(scale=1e-5,size=x_train1.shape[0])

In [29]:
x_exac_train = np.vstack((x_exac_train1, x_exac_train2))
y_exac_train = np.hstack((y_exac_train1, y_exac_train2))

In [30]:
train_lr(x_exac_train, y_exac_train)

Score: 1.0
coefficients: [0. 0. 0. 0.]


LinearRegression()

In [31]:
input_size = x_exac_train.shape[-1]
hidden_size = 20
output_size = 1
model_exac = RegressionNet(input_size, hidden_size, output_size)
train_net(model_exac, x_exac_train, y_exac_train)

epoch  10  loss =  0.004262304399162531
epoch  20  loss =  0.003943100571632385
epoch  30  loss =  0.0036478813271969557
epoch  40  loss =  0.0033748343121260405
epoch  50  loss =  0.0031222670804709196
epoch  60  loss =  0.002888636663556099
epoch  70  loss =  0.0026725162751972675
epoch  80  loss =  0.002472585067152977
epoch  90  loss =  0.0022876146249473095
epoch  100  loss =  0.0021164834033697844


In [32]:
print_loss(model_exac, *to_torch(x_exac_train3, y_exac_train3))

0.0009758505038917065


Now, let's create new samples from different ODE: $y'' = -y$. its exact solution is:
$ y(x) = c_1cos(x) + c_2sin(x) $

In [33]:
def harmonic_oscillator(x, c1, c2):
    return c1*np.cos(x) + c2*np.sin(x)

In [35]:
x_ho = np.sort(np.random.random(1000) * 10)
c1, c2 = 1, 1
y_ho = [harmonic_oscillator(x_i, c1, c2) for x_i in x_ho]

In [43]:
x_ho_d = gather_derivatives(x_ho, y_ho)
y_ho_d = np.zeros(x_ho_d.shape[0])

In [44]:
print_loss(model_exac, *to_torch(x_ho_d, y_ho_d))

4393076736.0
